In [1]:
from pyspark.sql import SparkSession  
import pyspark.sql.functions as func
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, BooleanType, DoubleType
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# Create a SparkSession
spark = (SparkSession
  .builder
  .config("spark.submit.deployMode", "client")
  .appName("SparkSQLExampleApp")
  .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/11 09:17:00 INFO SparkEnv: Registering MapOutputTracker
24/12/11 09:17:00 INFO SparkEnv: Registering BlockManagerMaster
24/12/11 09:17:00 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/12/11 09:17:00 INFO SparkEnv: Registering OutputCommitCoordinator


In [2]:
datafile='log20170112.csv'
schema = StructType([
    StructField("ip", StringType(), False),
    StructField("date", StringType(), False),
    StructField("time", StringType(), False),
    StructField("zone", DoubleType(), True),
    StructField("cik", DoubleType(), False),
    StructField("accession", StringType(), False),
    StructField("extention", StringType(), True),
    StructField("code", DoubleType(), False),
    StructField("size", DoubleType(), False),
    StructField("idx", DoubleType(), False),
    StructField("norefer", DoubleType(), True),
    StructField("noagent", DoubleType(), True),
    StructField("find", DoubleType(), True),
    StructField("crawler", DoubleType(), True),
    StructField("browser", StringType(), True)
])

In [3]:
df = (spark.read.format("csv")
  .schema(schema)
  .option("header", "true")
  .load(datafile))

In [4]:
df=df.withColumn("timestamp", concat(df.date, lit(" "), df.time).cast("Timestamp"))
df=df.drop("date").drop("time").drop("zone").drop("norefer").drop("browser")
df=df.where(df.code>199.0).where(df.noagent<=1.0).dropna()

In [5]:
df=df.withColumn("cik",df.cik.cast("Integer")).withColumn("code",df.code.cast("Integer")).withColumn("size",df.size.cast("Integer")).withColumn("idx",df.idx.cast("Boolean")).withColumn("noagent",df.noagent.cast("Boolean")).withColumn("crawler",df.crawler.cast("Boolean")).withColumn("find",df.find.cast("Integer"))
df=df.withColumn("webpage", concat(df.cik,lit("/"),df.accession,lit("/"),df.extention))
splitcol = split(df.extention,"\.")
df2 = df.withColumn("sc",splitcol).withColumn("al",size(splitcol))
df = df2.where(df2.al==2).withColumn("filetype", element_at(df2.sc,-1)).drop("sc").drop("al")
valid_filetypes = ["htm","txt","xml","xsd","css","pdf","html","zip","xls","xlsx","fil","xm","FIL","XML","png","asp","json","xsl","PDF","khtml"]
df = df.filter(df.filetype.isin(valid_filetypes))
df

DataFrame[ip: string, cik: int, accession: string, extention: string, code: int, size: int, idx: boolean, noagent: boolean, find: int, crawler: boolean, timestamp: timestamp, webpage: string, filetype: string]

In [6]:
df.show(10,False)

+--------------+-------+--------------------+----------+----+-----+-----+-------+----+-------+-------------------+---------------------------------------+--------+
|ip            |cik    |accession           |extention |code|size |idx  |noagent|find|crawler|timestamp          |webpage                                |filetype|
+--------------+-------+--------------------+----------+----+-----+-----+-------+----+-------+-------------------+---------------------------------------+--------+
|107.22.225.dea|1403095|0001209191-17-003373|-index.htm|200 |7483 |true |false  |10  |false  |2017-01-12 00:00:00|1403095/0001209191-17-003373/-index.htm|htm     |
|108.91.91.hbc |1694171|0001209191-17-003442|.txt      |301 |630  |false|false  |10  |false  |2017-01-12 00:00:00|1694171/0001209191-17-003442/.txt      |txt     |
|108.91.91.hbc |1134620|0001082038-17-000002|.txt      |200 |10022|false|false  |10  |false  |2017-01-12 00:00:00|1134620/0001082038-17-000002/.txt      |txt     |
|108.91.91.hbc |

In [7]:
#Number of logs
df.count()

15262098

In [8]:
# Most requested file types
df_mrft = df.groupBy("filetype").count().sort("count",ascending=False)
df_mrft.show(10)
dfp_mrft = df_mrft.limit(10).toPandas()

+--------+-------+
|filetype|  count|
+--------+-------+
|     htm|6928628|
|     txt|5845582|
|     xml|2001638|
|     xsd| 159120|
|     pdf| 136998|
|    html| 118720|
|     zip|  21098|
|     xls|  19891|
|    xlsx|  18996|
|     fil|   4984|
+--------+-------+
only showing top 10 rows



In [9]:
# Busiest time of day (requests)
df_btod = df.groupBy(window(df["timestamp"],"5 minute")).count()
df_btod = df_btod.withColumn("sttime", df_btod.window.start).drop("window").select("sttime","count")
df_btod = df_btod.sort("count",ascending=False).withColumnRenamed("count","requests")
dfp_btod = df_btod.limit(3).toPandas()

+-------------------+--------+
|             sttime|requests|
+-------------------+--------+
|2017-01-12 05:35:00|  154099|
|2017-01-12 05:40:00|  151292|
|2017-01-12 02:00:00|  147459|
|2017-01-12 01:50:00|  143385|
|2017-01-12 01:45:00|  142233|
|2017-01-12 02:05:00|  140348|
|2017-01-12 02:10:00|  138621|
|2017-01-12 02:15:00|  134946|
|2017-01-12 01:55:00|  133494|
|2017-01-12 05:30:00|  131333|
+-------------------+--------+
only showing top 10 rows



In [10]:
# Busiest time of day (request size)
df_btods = df.groupBy(window(df["timestamp"],"5 minute")).agg({"size":"sum"})
df_btods = df_btods.withColumn("sttime", df_btods.window.start).drop("window").withColumnRenamed("sum(size)","size").select("sttime","size")
df_btods = df_btods.sort("size",ascending=False)
dfp_btods = df_btods.limit(3).toPandas()

+-------------------+-----------+
|             sttime|       size|
+-------------------+-----------+
|2017-01-12 15:15:00|20472062503|
|2017-01-12 13:40:00|16627818367|
|2017-01-12 14:10:00|16528630962|
|2017-01-12 14:05:00|15856610105|
|2017-01-12 13:35:00|15772565588|
|2017-01-12 15:00:00|15336715121|
|2017-01-12 14:15:00|15048801224|
|2017-01-12 13:45:00|14948809685|
|2017-01-12 13:20:00|14863804923|
|2017-01-12 15:10:00|14709649782|
+-------------------+-----------+
only showing top 10 rows



In [11]:
# Most accessed webpage
df_maw = df.groupBy("webpage").count()
df_maw = df_maw.sort("count",ascending=False).limit(10)
dfp_maw = df_maw.limit(3).toPandas()

+--------------------+-----+
|             webpage|count|
+--------------------+-----+
|1429496/000000000...|99251|
|1565979/000000000...|98154|
|1429496/000000000...|97551|
|1584509/000158450...|35764|
|1584509/000158450...|35668|
|1584509/000158450...|35666|
|1584509/000158450...|35542|
|1584509/000158450...|35515|
|1584509/000158450...|35224|
|799288/0000799288...|19221|
+--------------------+-----+



In [12]:
# Most accessed non HTML asset
#find unique filetypes
df_manha = df.where(df.filetype!="htm").where(df.filetype!="html").groupBy("webpage").count()
df_manha = df_manha.sort("count",ascending=False).limit(10)
df_manha.show(10)

+--------------------+-----+
|             webpage|count|
+--------------------+-----+
|1429496/000000000...|99251|
|1429496/000000000...|97551|
|1584509/000158450...|35764|
|1584509/000158450...|35668|
|1584509/000158450...|35666|
|1584509/000158450...|35542|
|1584509/000158450...|35515|
|1584509/000158450...|35224|
|799288/0000799288...|19221|
|799288/0000799288...|19215|
+--------------------+-----+



In [13]:
# Largest document accessed
df_lda = df.sort("size",ascending=False).limit(10).select("webpage","size")
dfp_lda = df_lda.limit(3).toPandas()

+--------------------+---------+
|             webpage|     size|
+--------------------+---------+
|4904/0000004904-1...|434945787|
|6879/0000004904-1...|434914040|
|4904/0000004904-1...|434913884|
|1427437/000006598...|434412985|
|1427437/000006598...|422721117|
|4904/0000004904-1...|274817311|
|6879/0000004904-1...|274814011|
|4904/0000004904-1...|274813951|
|1528059/000150738...|270242348|
|1507385/000150738...|270222536|
+--------------------+---------+



In [14]:
# HTML response codes distribution
df_hrcd = df.groupBy("code").count()
dfp_hrcd = df_hrcd.toPandas()

In [15]:
# HTML response codes distribution over time
df_hrcdt = df.groupBy([window(df["timestamp"],"5 minute"),"code"]).count()
df_hrcdt = df_hrcdt.withColumn("sttime", df_hrcdt.window.start).drop("window").select("sttime","code","count")
dfp_hrcdt = df_hrcdt.toPandas()

In [16]:
# Requests histogram
df_rh = df_btod.sort("sttime")
dfp_rh = df_rh.toPandas()

In [17]:
# Request size histogram
df_rsh = df_btods.sort("sttime")
dfp_rsh = df_rsh.toPandas()

In [18]:
# Index page requests histogram
df_irh = df.where(df.idx==True).groupBy(window(df["timestamp"],"5 minute")).count()
df_irh = df_irh.withColumn("sttime", df_irh.window.start).drop("window").select("sttime","count")
df_irh = df_irh.sort("sttime")
dfp_irh = df_irh.toPandas()

In [19]:
# Most frequently requested index page
df_mfir = df.where(df.idx==True).groupBy("webpage").count()
df_mfir = df_mfir.sort("count",ascending=False)
df_mfir.show(10)

+--------------------+-----+
|             webpage|count|
+--------------------+-----+
|1565979/000000000...|98154|
|1067983/000120919...|18123|
|1078511/000120919...|16363|
|312069/9999999997...|14798|
|1590750/999999999...| 5401|
|1524025/000119312...| 5123|
|1310067/000119312...| 5083|
|1512499/000089706...| 5012|
|1305773/000130577...| 5001|
|1548777/000119312...| 4931|
+--------------------+-----+
only showing top 10 rows



In [20]:
# Crawler request histogram
df_crh = df.where(df.crawler==True).groupBy(window(df["timestamp"],"5 minute")).count()
df_crh = df_crh.withColumn("sttime", df_crh.window.start).drop("window").select("sttime","count")
df_crh = df_crh.sort("sttime")
dfp_crh = df_crh.toPandas()

In [21]:
# Crawler request size in comparison to others
df_crsc = df.groupBy("crawler").agg({"size":"sum"}).withColumnRenamed("sum(size)","size")
dfp_crsc = df_crsc.toPandas()

In [22]:
# Requests by source
df_rs = df.groupBy("find").agg({"find":"count"}).withColumnRenamed("count(find)","count")
df_rs = df_rs.sort("find")
dfp_rs = df_rs.toPandas()

In [42]:
# Sources by time
df_st = df.groupBy([window(df["timestamp"],"5 minute"),"find"]).count()
df_st = df_st.withColumn("sttime", df_st.window.start).drop("window").select("sttime","find","count").sort(["sttime","find","count"])
dfp_st = df_st.toPandas()

In [24]:
# Most Requests by IP Address
df_rip = df.groupBy("ip").count()
df_rip = df_rip.sort("count",ascending=False).limit(10)
dfp_rip = df_rip.limit(3).toPandas()

+---------------+-------+
|             ip|  count|
+---------------+-------+
|   24.61.85.aef|2061037|
|  54.152.17.ccg| 948106|
| 68.180.231.abf| 409767|
|  52.45.218.ihf| 343890|
|  108.91.91.hbc| 303159|
| 64.140.243.cgg| 296288|
|  117.89.50.ihd| 280503|
| 161.253.85.gch| 255056|
|  128.250.0.eda| 231266|
|217.174.255.dgd| 200537|
+---------------+-------+



In [52]:
# Largest requests by IP address
df_lrip = df.groupBy("ip").agg({"size":"sum"}).withColumnRenamed("sum(size)","size")
df_lrip = df_lrip.sort("size",ascending=False)
dfp_lrip = df_lrip.limit(3).toPandas()

'               ip          size\n0   163.117.2.gai  126336833930\n1  141.24.186.haa  122514343800\n2  128.103.40.bah   85604387541'

In [26]:
# Most popular CIKs
df_mpcik = df.groupBy("cik").count()
df_mpcik = df_mpcik.sort("count",ascending=False)
dfp_mpcik = df_mpcik.limit(3).toPandas()

+----------+------+
|       cik| count|
+----------+------+
|   1584509|214825|
|   1429496|196811|
|2147483647|147564|
|     95521|133394|
|    799288|115758|
|   1565979| 98158|
|   1166388| 95571|
|    906304| 86925|
|     19617| 70361|
|    935703| 58657|
+----------+------+
only showing top 10 rows



In [27]:
# Largest CIK requests
df_lcik = df.groupBy("cik").agg({"size":"sum"}).withColumnRenamed("sum(size)","size")
df_lcik = df_lcik.sort("size",ascending=False)
dfp_lcik = df_lcik.limit(3).toPandas()

+-------+-----------+
|    cik|       size|
+-------+-----------+
|1409970|20849222246|
|1331971|20645887549|
|1129987|14906707236|
| 933691|13906101611|
|1584509| 8503688654|
|1429496| 6850463109|
| 810893| 6154934896|
|   4904| 4700967317|
| 831001| 4322647271|
|1382911| 4288847321|
+-------+-----------+
only showing top 10 rows



In [82]:
dl = {"Attribute": ["IP Address","","","Central Index Key (CIK)","","","Webpage","","","Time of day","","",],
      "":[1,2,3,1,2,3,1,2,3,1,2,3],
      "Most Requests":[dfp_rip["ip"][0]+str(dfp_rip["count"][0]),dfp_rip["ip"][1]+str(dfp_rip["count"][1]),dfp_rip["ip"][2]+str(dfp_rip["count"][2]),
                       dfp_mpcik["cik"][0]+str(dfp_mpcik["count"][0]),dfp_mpcik["cik"][1]+str(dfp_mpcik["count"][1]),dfp_mpcik["cik"][2]+str(dfp_mpcik["count"][2]),
                       dfp_maw["webpage"][0]+str(dfp_maw["count"][0]),dfp_maw["webpage"][1]+str(dfp_maw["count"][1]),dfp_maw["webpage"][2]+str(dfp_maw["count"][2]),
                       dfp_btod["sttime"][0]+str(dfp_btod["requests"][0]),dfp_btod["sttime"][1]+str(dfp_btod["requests"][1]),dfp_btod["sttime"][2]+str(dfp_btod["requests"][2])],
      "Largest Requests":[dfp_lrip["ip"][0]+str(dfp_lrip["size"][0]),dfp_lrip["ip"][1]+str(dfp_lrip["size"][1]),dfp_lrip["ip"][2]+str(dfp_lrip["size"][2]),
                           dfp_lcik["cik"][0]+str(dfp_lcik["size"][0]),dfp_lcik["cik"][1]+str(dfp_lcik["size"][1]),dfp_lcik["cik"][2]+str(dfp_lcik["size"][2]),
                           dfp_lda["webpage"][0]+str(dfp_lda["size"][0]),dfp_lda["webpage"][1]+str(dfp_lda["size"][1]),dfp_lda["webpage"][2]+str(dfp_lda["size"][2]),
                           dfp_btods["sttime"][0]+str(dfp_btods["size"][0]),dfp_btods["sttime"][1]+str(dfp_btods["size"][1]),dfp_btods["sttime"][2]+str(dfp_btods["size"][2])]}
dfp_daily = pd.DataFrame(dl)

In [83]:
# Create dashboard
# Initialize the app
app = Dash()

# App layout
#app.layout = [
#    html.Div(children='My First App with Data'),
#    dash_table.DataTable(data=df.to_dict('records'), page_size=10),
#    dcc.Graph(figure=px.pie(df,"ip","size"))
#]
app.layout = html.Div([
    dcc.Tabs(id="tabs", value='daily', children=[
        dcc.Tab(label='Dashboard', value='daily')
    ]),
    html.Div(id='tab-content')
])

@app.callback(Output('tab-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
        return html.Div([
            html.H3('Server log analyses'),
            html.Div([
                dcc.Graph(figure=px.pie(dfp_mrft, names='filetype', values='count', title='Filetype request'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.pie(dfp_hrcd, names='code', values='count', title='HTML Response code'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.pie(dfp_crsc, names='crawler', values='size', title='Crawler traffic share'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.pie(dfp_rs, names='find', values='count', title='Request sources'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.bar(dfp_rh, x='sttime', y='requests', title='Requests by time of day'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.bar(dfp_rsh, x='sttime', y='size', title='Request size by time of day'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.bar(dfp_irh, x='sttime', y='count',  title='Index page requests by time of day'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.bar(dfp_crh, x='sttime', y='count', title='Crawler requests by time of day'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.line(dfp_st, x='sttime', y='count', color='find', title='Request source by time of day'))
            ], style={'width': '48%', 'display': 'inline-block','border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dcc.Graph(figure=px.bar(dfp_hrcdt, x='sttime', y='count', color='code', title='HTML Response codes by time of day'))
            ], style={'width': '48%', 'display': 'inline-block', 'border':'1px solid black', 'margin': '2px'}),
            html.Div([
                dash_table.DataTable(
                    data=dfp_daily.to_dict('records'), page_size=14
                )
            ])
        ])


#app.run(jupyter_mode="external", port=8050)

In [84]:
app.run(jupyter_mode="external", port=8050)

Dash app running on http://127.0.0.1:8050/
